https://www.youtube.com/watch?v=eHfMCtlsb1o

In [8]:
import ollama
from pydantic import BaseModel
from typing import List
import pickle as pkl

Primero tenemos que leer el fichero badbunny

In [9]:
import ollama
import bs4
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import TextLoader
from langchain_community.vectorstores import Chroma
from langchain_community.embeddings import OllamaEmbeddings
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

loader = TextLoader("bad_bunny.txt")
docs = loader.load()
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
splits = text_splitter.split_documents(docs)

In [10]:
splits

[Document(page_content='Benito Antonio Martínez Ocasio (Vega Baja, Puerto Rico 10 de marzo de 1994),2\u200b conocido artísticamente como Bad Bunny, es un cantante, compositor, productor musical y peleador aficionado de lucha libre puertorriqueño.3\u200b Sus estilos musicales son generalmente definidos como reguetón, trap latino y baladas, aunque también ha interpretado otros géneros y estilos variados. Se caracteriza por su entonación grave y su estilo de vestir.4\u200b', metadata={'source': 'bad_bunny.txt'}),
 Document(page_content='Empezó a ganar popularidad en SoundCloud y finalmente, firmó un contrato con el sello discográfico Hear This Music mientras trabajaba como empaquetador en un supermercado y estudiaba en la Universidad de Puerto Rico en Arecibo. Después del éxito de su sencillo «Soy peor» en 2016, alcanzó la fama tras colaborar con los artistas Cardi B y Drake en los sencillos «I Like It» y «Mia» que alcanzaron el primer y tercer puesto en la lista Billboard Hot 100 respect

Cargamos los embeddings

In [11]:
# Create Ollama embeddings and vector store
embedding_model = OllamaEmbeddings(model="llama2")

In [12]:
persist_directory = "embeddings"
vectorstore = Chroma.from_documents(documents=splits, embedding=embedding_model, persist_directory=persist_directory)

In [13]:
# Create the retriever
retriever = vectorstore.as_retriever()

def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

# Define the Ollama LLM function
def ollama_llm(question, context):
    formatted_prompt = f"Question: {question}\n\nContext: {context}"
    response = ollama.chat(model='llama2', messages=[{'role': 'user', 'content': formatted_prompt}])
    return response['message']['content']

# Define the RAG chain
def rag_chain(question):
    retrieved_docs = retriever.invoke(question)
    print(retrieved_docs)
    formatted_context = format_docs(retrieved_docs)
    return ollama_llm(question, formatted_context)

# Use the RAG chain
result = rag_chain("Album de Bad Bunny por el que ganó un Grammy Latino en 2023.")
print(result)

[Document(page_content='Bad Bunny es reconocido por su gran variedad de looks.', metadata={'source': 'bad_bunny.txt'}), Document(page_content='Durante su adolescencia, pasó por "temporadas musicales" en las que escuchaba casi exclusivamente un determinado tipo de música; él reflexiona que "hubo temporada de bachata, temporada de indie-pop, incluso temporada de Bee Gees".19\u200b Daddy Yankee es una gran influencia para Bad Bunny, pero Lavoe es su mayor inspiración y se describe como "una hebra fundamental del ADN musical de Bad Bunny";107\u200b Luego dijo que, cuando crecía, entretenía a los vecinos desde su balcón interpretando canciones de Lavoe.19\u200b Aunque es principalmente un artista de trap y reguetón, sus canciones también incluyen influencias del soul, el pop y el R&B.28\u200b Abordó su experimentación musical afirmando: "Si mañana quiero sacar un disco de rock o uno de bachata, nadie me puede decir nada, ¿por qué yo no? Hay que intentar unir públicos, unir países, unir gust

Aqui podemos ver que sin context el modelo no responde bien.

In [14]:
ollama.chat(model='llama2', messages=[{'role': 'user', 'content': "Album de Bad Bunny por el que ganó un Grammy Latino en 2023."}])

{'model': 'llama2',
 'created_at': '2024-04-17T20:57:20.562215Z',
 'message': {'role': 'assistant',
  'content': '\nEl álbum de Bad Bunny por el que ganó un Grammy Latino en 2023 es "YHLQMDLG". Este álbum fue lanzado en 2019 y obtuvo una nominación en la categoría de Mejor Album de Musica Urbana en los Premios Grammy Latinos de 2020, pero no ganó. Sin embargo, en 2023, Bad Bunny ganó el premio a Mejor Album de Musica Urbana por "YHLQMDLG" en la edición número 24 de los Premios Grammy Latinos.'},
 'done': True,
 'total_duration': 13681915167,
 'load_duration': 41262167,
 'prompt_eval_count': 28,
 'prompt_eval_duration': 329768000,
 'eval_count': 137,
 'eval_duration': 13244032000}

Ahora queremos que responda con un JSON

https://www.youtube.com/watch?v=eHfMCtlsb1o

In [15]:
from openai import OpenAI
from pydantic import BaseModel, Field
from typing import List
import instructor

In [16]:
class BadBunnyInfo(BaseModel):
    artist_name: str = Field(..., description="Artist name of the singer")
    name: str = Field(..., description="Real Name")
    surname: str = Field(..., description="Real Surname")
    birth_date: str = Field(..., description="Birth Date")
    year_YHLQMDLG: int = Field(..., description="Year of the album YHLQMDLG")

In [17]:
client = instructor.patch(
    OpenAI(
        base_url="http://localhost:11434/v1",
        api_key="ollama",
    ),
    mode=instructor.Mode.JSON,
)


Primero cogemos solo la parte donde esta la informacion.

In [18]:
queries = [
"Nombre artistico del cantante de Bad Bunny",
"Como se llama el cantante Bad Bunny",
"Apellidos del cantante Bad Bunny",
"Fecha de Nacimiento del cantante Bad Bunny",
"En que año lanzo su album YHLQMDLG",
]


In [19]:
retrieved_docs = {}
for query in queries:
    retrieved_doc = retriever.invoke(query)
    retrieved_docs[query] = retrieved_doc

In [20]:
result = ""

for query, doc in retrieved_docs.items():
    print(query)
    result += query
    print("\n")
    result += "\n"
    for page in doc:
        print(page.page_content)
        result += page.page_content
        print("\n")
        result += "\n"

Nombre artistico del cantante de Bad Bunny


2020-2021: YHLQMDLG, Las que no iban a salir y El último tour del mundo


que pasó el rapero en un Airbnb de Puerto Rico.19​ Bad Bunny se convirtió en el primer artista urbano latino en aparecer en la portada de la revista.16​ Además, las fotos realizadas para el reportaje fueron creadas por Gabriela Berlingeri, diseñadora de joyas y novia de Bad Bunny, lo que la convierte en la primera latina que realiza un reportaje de portada para la revista.16​ Con el reportaje, la editora de música latina de Rolling Stone, Suzy Exposito, también se convirtió en la primera latina en escribir un artículo de portada para la revista.16​


El disco fue lanzado el 6 de mayo de 2022.91​ Este cuarto álbum de estudio de Benito, consta de 23 sencillos, en los cuales abarco géneros como Reguetón, trap latino, rock, merengue, deep house, synth pop, dancehall, entre otros.92​ Cuenta con las colaboraciones de Chencho Corleone, Jhay Cortez, Tony Dize, Rauw Alejandro, 

In [21]:
retrieved_docs['Fecha de Nacimiento del cantante Bad Bunny']

[Document(page_content='2020-2021: YHLQMDLG, Las que no iban a salir y El último tour del mundo', metadata={'source': 'bad_bunny.txt'}),
 Document(page_content='que pasó el rapero en un Airbnb de Puerto Rico.19\u200b Bad Bunny se convirtió en el primer artista urbano latino en aparecer en la portada de la revista.16\u200b Además, las fotos realizadas para el reportaje fueron creadas por Gabriela Berlingeri, diseñadora de joyas y novia de Bad Bunny, lo que la convierte en la primera latina que realiza un reportaje de portada para la revista.16\u200b Con el reportaje, la editora de música latina de Rolling Stone, Suzy Exposito, también se convirtió en la primera latina en escribir un artículo de portada para la revista.16\u200b', metadata={'source': 'bad_bunny.txt'}),
 Document(page_content='Activismo\nEn 2019, fue nominado a los Premios Tu Música Urbano inaugurales de Telemundo, en la categoría de «Premio humanitario del Año».126\u200b\n\nEl 22 de julio de 2019, Bad Bunny se unió a arti

In [22]:
content = """
Hola me llamo Bad Bunny, mi nombre real es Benito Antonio Martínez Ocasio, nací el 10 de marzo de 1994 y mi album YHLQMDLG lo publiqué en 2020.
"""

# Write same content but in english
content = """
Hello my name is Bad Bunny, my real name is Benito Antonio Martínez Ocasio, I was born on March 10, 1994 and I released my album YHLQMDLG in 2020.
"""
# content = """
# Tienes en un texto la información de un cantante. El texto contiene información sobre su nombre real, su fecha de nacimiento y el año en el que su álbum YHLQMDLG. Aqui puedes encontrar el texto: 
# {texto}

# Por favor, responde a las siguientes preguntas:
# 1. Cual es su nombre artistico
# 2. Cual es su nombre real
# 3. Cual es su apellido
# 4. Cual es su fecha de nacimiento
# 5. En que año publico su album YHLQMDLG

# Responde con un JSON con keys
# 1. 'artirt_name'
# 2. 'name'
# 3. 'surname'
# 4. 'birth_date'
# 5. 'year_YHLQMDLG'
# """

In [23]:
resp = client.chat.completions.create(
    model="llama2",
    messages=[
        {
            "role": "user",
            #"content": content.format(texto=result)
            "content": content
        }
    ],
    response_model=BadBunnyInfo,
    max_retries=10
)

ValidationError: 5 validation errors for BadBunnyInfo
artist_name
  Field required [type=missing, input_value={'BadBunnyInfo': {'artist...G': 2020}, 'errors': []}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.7/v/missing
name
  Field required [type=missing, input_value={'BadBunnyInfo': {'artist...G': 2020}, 'errors': []}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.7/v/missing
surname
  Field required [type=missing, input_value={'BadBunnyInfo': {'artist...G': 2020}, 'errors': []}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.7/v/missing
birth_date
  Field required [type=missing, input_value={'BadBunnyInfo': {'artist...G': 2020}, 'errors': []}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.7/v/missing
year_YHLQMDLG
  Field required [type=missing, input_value={'BadBunnyInfo': {'artist...G': 2020}, 'errors': []}, input_type=dict]
    For further i

No funciona vamos a hacerlo mas simple